# Atividade Final de Inteligência Artificial

## Projeto: Classificação de Tabuleiros de Sudoku com LTN
---

**Aluno:** Antonio Mileysson França Bragança

**Matrícula:** 21850963

**Curso:** Engenharia da Computação

## Descrição do Notebook

Este notebook demonstra como utilizar **Redes Lógicas Tensoriais (Logic Tensor Networks - LTN)** para um problema de classificação de satisfatibilidade.

O objetivo é treinar modelos que sejam capazes de:

1. Classificar se um tabuleiro de Sudoku 4x4 preenchido(fechado) é **válido** ou **inválido**, ou seja, dizer se o tabuleiro estar corretamente preenchindo ou não.
2. Classificar um tabuleiro inicial/aberto.
3. Indicar para um tabuleiro aberto qual das heuristicas de resolução são mais recomendadas.

## 1\. Instalação de Dependências

Primeiro, vamos instalar as bibliotecas necessárias. `LTNtorch` é o framework principal que integra lógica fuzzy com PyTorch.

In [1]:
!pip install LTNtorch torch numpy pandas scikit-learn -q

## 2\.  Importações e Configuração Inicial

Nesta etapa, será importado as bibliotecas necessárias e  será configurado o ambiente para o treinamento do modelo.

In [2]:
import torch
import torch.nn as nn
import ltn
import numpy as np
import random
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Verifica se há GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo selecionado: {device}")

Dispositivo selecionado: cpu


## 3\. Geração de Dados de Treino
Para treinar o modelo, precisa-se de dados. Portanto, será gerado um conjunto de tabuleiros 4x4:

  * **Exemplos Positivos (Válidos)**: Tabuleiros completamente preenchidos que seguem todas as regras do Sudoku.
  * **Exemplos Negativos (Inválidos)**: Tabuleiros que violam pelo menos uma regra (ex: números repetidos em uma linha).

Será usado um algoritmo de *backtracking* para gerar soluções válidas e, em seguida, será introduzido erros para criar os exemplos inválidos.

In [3]:
# Função auxiliar para verificar se um número pode ser colocado em uma posição
def is_valid_placement(board, row, col, num):
    # Verifica a linha
    if num in board[row, :]:
        return False
    # Verifica a coluna
    if num in board[:, col]:
        return False
    # Verifica o bloco 2x2
    start_row, start_col = 2 * (row // 2), 2 * (col // 2)
    if num in board[start_row:start_row+2, start_col:start_col+2]:
        return False
    return True

# Função principal que preenche o tabuleiro usando backtracking
def solve_4x4(board):
    for row in range(4):
        for col in range(4):
            if board[row, col] == 0: # Encontra uma célula vazia
                nums = list(range(1, 5))
                random.shuffle(nums) # Tenta números em ordem aleatória
                for num in nums:
                    if is_valid_placement(board, row, col, num):
                        board[row, col] = num
                        if solve_4x4(board):
                            return True # Sucesso!
                        board[row, col] = 0 # Backtrack: desfaz a tentativa
                return False # Nenhuma opção funcionou
    return True # Todas as células preenchidas

def generate_valid_board():
    # 1. Cria um tabuleiro vazio
    board = np.zeros((4, 4), dtype=int)

    # 2. Preenche ele completamente usando o resolvedor
    solve_4x4(board)

    # 3. Converte para float e normaliza para o intervalo [0, 1]
    base = board.astype(np.float32)
    base = (base - 1) / 3

    return base.copy()

def generate_dataset(num_samples):
    dataset = []
    for _ in range(num_samples // 2):
        # Gerar um tabuleiro válido (label = 1)
        valid_board = generate_valid_board()
        dataset.append((valid_board, 1.0))

        # Gerar um tabuleiro inválido (label = 0)
        # Criar um tabuleiro válido e introduzir um erro aleatório
        invalid_board = generate_valid_board()
        row, col = random.randint(0, 3), random.randint(0, 3)
        # Garante que a célula não seja 0 (que já é inválido no contexto de um tabuleiro preenchido)
        while invalid_board[row, col] == 0:
             row, col = random.randint(0, 3), random.randint(0, 3)
        # Troca o valor de uma célula com o de outra na mesma linha (garante duplicata na linha)
        col_to_swap = random.choice([c for c in range(4) if c != col])
        invalid_board[row, col] = invalid_board[row, col_to_swap]

        dataset.append((invalid_board, 0.0))

    random.shuffle(dataset)
    return dataset

## 4. Definição do Modelo (SudokuLTN)
A classe `SudokuLTN` define a lógica do Sudoku usando LTN.
### Como funciona

* **Predicados aprendidos:**
  Pequenas redes neurais (`row_pred`, `col_pred`, `block_pred`) aprendem a reconhecer linhas, colunas e blocos válidos. A saída `Sigmoid` retorna um grau de verdade em `[0,1]`.

* **Conectivos lógicos:**

  * `And`: operador lógico **E** (combina condições).
  * `ForAll`: quantificador **para todo** (∀), aplicando a condição a todas as linhas, colunas e blocos.

* **Forward pass:**
  A rede constrói e avalia a seguinte fórmula:

  $$
  \forall r,\; P_{\text{row}}(r) \land
  \forall c,\; P_{\text{col}}(c) \land
  \forall b,\; P_{\text{block}}(b)
  $$

  O resultado é um **valor de verdade único** indicando o quão válido é o tabuleiro inteiro.


In [4]:
class SudokuLTN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Camadas neurais (os parâmetros que serão aprendidos)
        self.shared_layer = torch.nn.Sequential(
            torch.nn.Linear(4, 32),
            torch.nn.ReLU()
        )
        self.row_layer = torch.nn.Linear(32, 1)
        self.col_layer = torch.nn.Linear(32, 1)
        self.block_layer = torch.nn.Linear(32, 1)

        # Usando torch.nn.Sigmoid() para garantir a saída em [0, 1]
        self.row_pred = ltn.Predicate(
            torch.nn.Sequential(self.shared_layer, self.row_layer, torch.nn.Sigmoid())
        )
        self.col_pred = ltn.Predicate(
            torch.nn.Sequential(self.shared_layer, self.col_layer, torch.nn.Sigmoid())
        )
        self.block_pred = ltn.Predicate(
            torch.nn.Sequential(self.shared_layer, self.block_layer, torch.nn.Sigmoid())
        )

        # Definição dos operadores lógicos que serão usados
        self.ForAll = ltn.Quantifier(ltn.fuzzy_ops.AggregMin(), quantifier="f")
        self.And = ltn.Connective(ltn.fuzzy_ops.AndMin())

    def forward(self, board_tensor):
        board_tensor = board_tensor.to(device)

        # Preparar os dados para os quantificadores
        rows = board_tensor
        cols = board_tensor.T
        blocks = board_tensor.view(2, 2, 2, 2).permute(0, 2, 1, 3).contiguous().view(4, 4)

        # Definição da lógica declarativa
        x_row = ltn.Variable("rows", rows)
        y_col = ltn.Variable("cols", cols)
        z_block = ltn.Variable("blocks", blocks)

        all_rows_valid = self.ForAll(x_row, self.row_pred(x_row))
        all_cols_valid = self.ForAll(y_col, self.col_pred(y_col))
        all_blocks_valid = self.ForAll(z_block, self.block_pred(z_block))

        validity = self.And(all_rows_valid, self.And(all_cols_valid, all_blocks_valid))

        return validity # O resultado é um ltn.Tensor com o valor de verdade

## 5\. Treinamento do Modelo

Agora vamos treinar o modelo. O processo é semelhante ao treinamento padrão em PyTorch, mas com algumas adaptações para o contexto do LTN:

  - **Otimizador**: `Adam` para ajustar os pesos da rede.
  - **Função de Perda**: `MSELoss` (Mean Squared Error) para medir a diferença entre a saída do modelo e o rótulo verdadeiro (1.0 para válido, 0.0 para inválido).
  - **Early Stopping**: Uma técnica para interromper o treinamento se o desempenho no conjunto de validação não melhorar por um certo número de épocas, evitando overfitting.

In [5]:
def train():
    # ----- Preparação -----
    model = SudokuLTN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = torch.nn.MSELoss()

    # Geração de dataset (600 exemplos)
    dataset = generate_dataset(600)
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

    best_val_loss = float('inf')
    patience = 8
    no_improve = 0

    print(f"\n Iniciando treinamento no dispositivo: {device}")
    for epoch in range(70): # número de épocas
        model.train()
        train_loss = 0.0

        # embaralhar dados de treino a cada época
        random.shuffle(train_data)

        for board, label in train_data:
            optimizer.zero_grad()
            board_tensor = torch.tensor(board, dtype=torch.float32).to(device)
            target_tensor = torch.tensor(label, dtype=torch.float32).to(device)

            output = model(board_tensor)
            loss = loss_fn(output.value, target_tensor)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()

        # ----- Validação -----
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for board, label in val_data:
                board_tensor = torch.tensor(board, dtype=torch.float32).to(device)
                target_tensor = torch.tensor(label, dtype=torch.float32).to(device)

                output = model(board_tensor)
                val_loss += loss_fn(output.value, target_tensor).item()

        # médias
        train_loss /= len(train_data)
        val_loss /= len(val_data)

        # log mais frequente
        if (epoch + 1) % 5 == 0:
            print(f"Época : {epoch+1:02d} Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f}")

        # early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve = 0
            torch.save(model.state_dict(), "sudoku_ltn.pth")
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"\n Early stopping na época {epoch+1}")
                break

    print("\n Treinamento finalizado!")
    print(f"Melhor Val Loss obtida: {best_val_loss:.5f}")

train()


 Iniciando treinamento no dispositivo: cpu
Época : 05 Train Loss: 0.22813 | Val Loss: 0.22336
Época : 10 Train Loss: 0.19806 | Val Loss: 0.19744
Época : 15 Train Loss: 0.17431 | Val Loss: 0.18630
Época : 20 Train Loss: 0.16234 | Val Loss: 0.16303
Época : 25 Train Loss: 0.15359 | Val Loss: 0.15599
Época : 30 Train Loss: 0.14207 | Val Loss: 0.16192
Época : 35 Train Loss: 0.14295 | Val Loss: 0.15030
Época : 40 Train Loss: 0.14091 | Val Loss: 0.14102
Época : 45 Train Loss: 0.13963 | Val Loss: 0.13943
Época : 50 Train Loss: 0.13970 | Val Loss: 0.13960
Época : 55 Train Loss: 0.13910 | Val Loss: 0.14025
Época : 60 Train Loss: 0.13915 | Val Loss: 0.14090
Época : 65 Train Loss: 0.13882 | Val Loss: 0.13934
Época : 70 Train Loss: 0.13822 | Val Loss: 0.13997

 Treinamento finalizado!
Melhor Val Loss obtida: 0.13835


 ## 6\. Classificação um tabuleiro fechado

Com o modelo já treinado, podemos utilizá-lo para classificar novos tabuleiros de Sudoku. Isso significa que ele consegue determinar se um tabuleiro está corretamente preenchido (válido) ou se contém erros (inválido).

### Execução da Validação

Agora, carregamos o  modelo e as funções auxiliares para ler o CSV e validar cada tabuleiro.

In [6]:
def validate_sudoku(board, model, threshold=0.7):
    """
    Valida um tabuleiro de Sudoku 4x4 usando o modelo LTN treinado.
    """
    # Normalizar o tabuleiro de teste da mesma forma que os dados de treino
    normalized_board = (board - 1) / 3
    board_tensor = torch.tensor(normalized_board, dtype=torch.float32).to(device)

    with torch.no_grad():
        # Acessar o atributo .value antes de chamar .item()
        validity_prob = model(board_tensor).value.item()

    return validity_prob > threshold

#Função para ler os tabuleiros de um arquivo CSV
def load_closed_boards_from_csv(file_path):
    """
    Carrega tabuleiros de Sudoku e suas descrições de um arquivo CSV.
    """
    try:
        df = pd.read_csv(file_path)
        boards = []
        for index, row in df.iterrows():
            # Extrai os 16 valores do tabuleiro e os converte para float
            board_values = row.iloc[0:16].values.astype(np.float32)
            # Remodela a lista de valores para uma matriz 4x4
            board_array = np.reshape(board_values, (4, 4))
            boards.append(board_array)
        return boards
    except FileNotFoundError:
        print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo CSV: {e}")
        return []

# Carregar o modelo treinado
model = SudokuLTN().to(device)
model.load_state_dict(torch.load("sudoku_ltn.pth"))
model.eval()

# Executar a validação para cada tabuleiro lido do CSV
print("\n--- INICIANDO TESTES DE VALIDAÇÃO---")

test_boards = load_closed_boards_from_csv("sudoku_4x4_closed_boards.csv")

if not test_boards:
   print("Nenhum tabuleiro de teste foi carregado. Verifique o arquivo CSV.")
else:
  for board in test_boards:
        print(board)
        result = validate_sudoku(board, model)
        print("Resultado: Válido!" if result else "Resultado: Inválido!")
        print("-" * 20)



--- INICIANDO TESTES DE VALIDAÇÃO---
[[1. 2. 3. 4.]
 [3. 4. 1. 2.]
 [2. 1. 4. 3.]
 [4. 3. 2. 1.]]
Resultado: Válido!
--------------------
[[1. 4. 2. 3.]
 [2. 3. 1. 4.]
 [3. 2. 4. 1.]
 [4. 1. 3. 2.]]
Resultado: Válido!
--------------------
[[1. 2. 3. 4.]
 [1. 2. 3. 4.]
 [2. 1. 4. 3.]
 [4. 3. 2. 1.]]
Resultado: Inválido!
--------------------
[[1. 2. 3. 1.]
 [3. 4. 1. 2.]
 [2. 1. 4. 3.]
 [4. 3. 2. 4.]]
Resultado: Inválido!
--------------------
[[1. 1. 3. 4.]
 [2. 2. 1. 2.]
 [2. 1. 4. 3.]
 [4. 3. 2. 1.]]
Resultado: Inválido!
--------------------


## 7. Classificação de um tabuleiro inicial/aberto (usando heuristicas)

Agora, estenderemos a funcionalidade do modelo LTN treinado para uma tarefa  mais complexa: a análise de tabuleiros incompletos (abertos), com o objetivo de determinar os movimentos mais promissores para a sua resolução.

Esta abordagem é **híbrida** porque combina duas técnicas:

1.  **Heurísticas Clássicas**: Primeiro, usa-se regras lógicas para verificar se o tabuleiro já está em um estado sem solução. Por exemplo, se um número que ainda precisa ser jogado não pode ser colocado em nenhuma célula vazia, o jogo já está perdido. A classe `SudokuHeuristicAnalyzer` cuida dessa verificação inicial.

2.  **Intuição do Modelo LTN **: Se o tabuleiro passa na verificação heurística, usamos o modelo LTN treinado como um **avaliador de potencial**. Para cada movimento possível:

      * Simulamos o movimento em uma cópia do tabuleiro.
      * Pedimos ao modelo LTN para "pontuar" o quão "promissor" ou "próximo de uma solução válida" o novo tabuleiro parece.
      * As células vazias são preenchidas com um valor neutro (0.5 após a normalização) para que o modelo possa avaliá-las sem viés.

A classe `LTNMoveAnalyzer` orquestra esse processo, utilizando a `LTNClassifier` (uma interface para o modelo treinado) para pontuar todos os movimentos e classificá-los do mais seguro ao mais arriscado. O resultado é um guia estratégico que sugere qual jogada tem a maior probabilidade de levar a uma solução correta.

In [8]:
# =============================================================================
# CLASSES DE ANÁLISE
# =============================================================================
class SudokuHeuristicAnalyzer:
    """Classe que aplica heurísticas a um tabuleiro de Sudoku."""
    def __init__(self, board):
        self.board = np.array(board, dtype=int)

    def _has_forced_no_solution(self):
        """Verifica se o tabuleiro está em um estado onde não há mais solução possível."""
        empty_cells = list(zip(*np.where(self.board == 0)))

        # Se não há células vazias, não há como fazer jogadas
        if not empty_cells:
            return False

        # Verifica se algum número que ainda precisa ser colocado não tem lugar válido
        for num in range(1, 5):
            # Se um número já foi colocado 4 vezes, pulamos
            if np.count_nonzero(self.board == num) < 4:
                can_place = any(
                    is_valid_placement(self.board, r, c, num) for r, c in empty_cells
                )
                if not can_place:
                    return True
        return False

    def get_all_possible_moves(self):
        """Retorna uma lista de todos os movimentos válidos (tupla: r, c, num)."""
        moves = []
        for r, c in list(zip(*np.where(self.board == 0))):
            for num in range(1, 5):
                if is_valid_placement(self.board, r, c, num):
                    moves.append((r, c, num))
        return moves


class LTNClassifier:
    """Interface para usar o modelo LTN treinado como um classificador"""
    def __init__(self, model_path):
        self.device = device
        self.model = SudokuLTN().to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()

    def _prepare_board_for_model(self, board):
        """Prepara um tabuleiro (potencialmente com zeros) para o modelo LTN."""
        # O valor 2.5 é o ponto médio do intervalo original [1, 4].
        # Quando normalizado (x-1)/3, 2.5 se torna 0.5, representando um valor neutro
        # para as células vazias, ideal para a avaliação do modelo.
        NEUTRAL_FILL_VALUE = 2.5
        prepared_board = board.copy().astype(np.float32)
        prepared_board[prepared_board == 0] = NEUTRAL_FILL_VALUE
        normalized_board = (prepared_board - 1) / 3.0
        return torch.tensor(normalized_board, dtype=torch.float32).to(self.device)

    def score_board_state(self, board):
        """Avalia um tabuleiro e retorna a pontuação de 'validade' do modelo."""
        with torch.no_grad():
            return self.model(self._prepare_board_for_model(board)).value.item()


class LTNMoveAnalyzer:
    """Classe principal que orquestra a análise híbrida de movimentos"""
    def __init__(self, board, ltn_classifier):
        self.board = np.array(board, dtype=int)
        self.classifier = ltn_classifier
        self.helper_analyzer = SudokuHeuristicAnalyzer(self.board)

    def classify_and_analyze(self):
        """Executa a análise completa e imprime os resultados."""
        print("=" * 60)
        print("Classificando o tabuleiro com o Modelo LTN:")
        print(self.board)
        print("=" * 60)

        if self.helper_analyzer._has_forced_no_solution():
            print("CLASSIFICAÇÃO: (1) Sem Solução")
            return

        print("CLASSIFICAÇÃO: (2) Solução Possível")
        possible_moves = self.helper_analyzer.get_all_possible_moves()
        if not possible_moves:
            print("O tabuleiro já está completo ou não há mais movimentos possíveis.")
            return

        # Avalia cada movimento possível usando o modelo LTN
        move_scores = {
            move: self.classifier.score_board_state(self.board_after_move(move))
            for move in possible_moves
        }
        sorted_moves = sorted(move_scores.items(), key=lambda item: item[1], reverse=True)

        print("\n--- Análise de Movimentos (Probabilidade de levar a uma solução) ---")
        print("a. Análise com UM (1) movimento:")
        print("   Movimentos com maior probabilidade (do melhor para o pior):")
        for move, score in sorted_moves:
            r, c, num = move
            print(f"      -> Jogar {num} em ({r}, {c}) | Pontuação : {score:.4f}")

        print("\nb. Análise com DOIS (2) movimentos:")
        best_first_move, best_score = sorted_moves[0]
        worst_first_move, worst_score = sorted_moves[-1]

        br, bc, bnum = best_first_move
        wr, wc, wnum = worst_first_move

        print(
            f"    - O movimento inicial mais seguro é Jogar {bnum} em ({br}, {bc}) com pontuação {best_score:.5f}."
        )
        print(
            f"    - O movimento inicial mais arriscado é Jogar {wnum} em ({wr}, {wc}) com pontuação {worst_score:.5f}."
        )
        print("=" * 60 + "\n")

    def board_after_move(self, move):
        r, c, num = move
        next_board = self.board.copy()
        next_board[r, c] = num
        return next_board


# =============================================================================
# FUNÇÃO PARA LEITURA DE TABULEIROS EM CSV
# =============================================================================
def load_open_boards_from_csv(file_path):
    """
    Lê um ou mais tabuleiros de Sudoku 4x4 de um arquivo CSV.
    Cada 4 linhas no arquivo são tratadas como um tabuleiro.
    """
    try:
        # Lê o arquivo CSV para um DataFrame do pandas.
        # header=None indica que não há linha de cabeçalho.
        df = pd.read_csv(file_path, header=None)

        # Verifica se o número de linhas é um múltiplo de 4
        if len(df) % 4 != 0:
            print(
                f"AVISO: O arquivo '{file_path}' tem {len(df)} linhas, que não é um múltiplo de 4. "
                "Tabuleiros incompletos no final do arquivo serão ignorados."
            )

        todos_os_tabuleiros = []
        # Itera sobre o DataFrame em blocos de 4 linhas
        for i in range(0, len(df), 4):
            # Garante que há um bloco completo de 4 linhas para formar um tabuleiro
            if i + 4 <= len(df):
                bloco_tabuleiro = df.iloc[i:i + 4]
                tabuleiro_lista = bloco_tabuleiro.values.tolist()
                todos_os_tabuleiros.append(tabuleiro_lista)

        return todos_os_tabuleiros

    except FileNotFoundError:
        print(f"ERRO: O arquivo '{file_path}' não foi encontrado.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo com pandas: {e}")
        return []


# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================
try:
    # Passo 1: Carrega o classificador LTN.
    ltn_classifier = LTNClassifier("sudoku_ltn.pth")

    # Passo 2: Define o nome do arquivo e lê os tabuleiros a partir dele.
    nome_do_arquivo_csv = "sudoku_4x4_open_boards.csv"
    tabuleiros = load_open_boards_from_csv(nome_do_arquivo_csv)

    if not tabuleiros:
        print("Nenhum tabuleiro foi carregado. Verifique o arquivo CSV ou o caminho.")
    else:
        print(f"\n{len(tabuleiros)} tabuleiro(s) lido(s) de '{nome_do_arquivo_csv}'.")

        # Passo 3: Itera sobre cada tabuleiro lido e o analisa.
        for i, tabuleiro in enumerate(tabuleiros):
            analyzer = LTNMoveAnalyzer(tabuleiro, ltn_classifier)
            analyzer.classify_and_analyze()

except (FileNotFoundError, NameError) as e:
    print(
        "\nERRO NA EXECUÇÃO: Certifique-se de que a célula de treinamento foi executada "
        "e o arquivo 'sudoku_ltn.pth' foi criado."
    )



2 tabuleiro(s) lido(s) de 'sudoku_4x4_open_boards.csv'.
Classificando o tabuleiro com o Modelo LTN:
[[1 0 0 4]
 [0 0 1 0]
 [0 1 0 0]
 [4 0 0 1]]
CLASSIFICAÇÃO: (2) Solução Possível

--- Análise de Movimentos (Probabilidade de levar a uma solução) ---
a. Análise com UM (1) movimento:
   Movimentos com maior probabilidade (do melhor para o pior):
      -> Jogar 3 em (0, 1) | Pontuação : 0.0067
      -> Jogar 3 em (1, 1) | Pontuação : 0.0067
      -> Jogar 4 em (1, 1) | Pontuação : 0.0067
      -> Jogar 3 em (3, 1) | Pontuação : 0.0067
      -> Jogar 3 em (0, 2) | Pontuação : 0.0061
      -> Jogar 2 em (1, 0) | Pontuação : 0.0061
      -> Jogar 3 em (1, 0) | Pontuação : 0.0061
      -> Jogar 2 em (1, 3) | Pontuação : 0.0061
      -> Jogar 3 em (1, 3) | Pontuação : 0.0061
      -> Jogar 2 em (2, 0) | Pontuação : 0.0061
      -> Jogar 3 em (2, 0) | Pontuação : 0.0061
      -> Jogar 3 em (2, 2) | Pontuação : 0.0061
      -> Jogar 4 em (2, 2) | Pontuação : 0.0061
      -> Jogar 2 em (2, 3) |

## 8. LTN como Recomendador de Heurísticas de Solução

Nesta seção, elevamos o nível de abstração. Em vez de perguntar ao modelo "qual é o melhor próximo movimento?", vamos perguntar: "**Qual é a melhor *estratégia* para encontrar o próximo movimento?**"

Para isso, será construído um **Recomendador de Heurísticas**. Humanos que resolvem Sudokus difíceis não usam apenas força bruta; eles reconhecem padrões e aplicam técnicas específicas. O novo modelo será ensinado a fazer o mesmo, recomendando uma das seguintes estratégias com base no estado atual do tabuleiro:

1.  **Naked Single (Candidato Único):** A heurística mais simples. Ocorre quando uma célula vazia tem apenas um número possível que pode ser colocado nela.
2.  **Hidden Single (Único Escondido):** Mais sutil. Ocorre quando, dentro de uma linha, coluna ou bloco, um número específico só pode ser colocado em uma única célula, mesmo que essa célula tenha outros candidatos.
3.  **Locked Candidate (Candidato Travado):** Uma técnica avançada. Ocorre quando os candidatos de um número dentro de um bloco estão confinados a uma única linha ou coluna, permitindo eliminar esse candidato do resto da linha ou coluna fora do bloco.
4.  **Backtracking (Último Recurso):** Se nenhuma das heurísticas mais simples se aplica, a estratégia restante é a de tentativa e erro.

Para alcançar isso, Esse novo modelo, `LTNHeuristicRecommender4x4`, usará:

  * **Engenharia de Features:** Em vez de apenas os valores do tabuleiro, forneceremos um "tensor de features" com 3 canais de informação para cada célula: o valor da célula, a contagem de candidatos possíveis e uma camada de conflitos.
  * **Predicados Especialistas:** O modelo terá predicados LTN separados, cada um treinado para se tornar um "especialista" em detectar o potencial de uma das heurísticas (ex: `NakedSinglePotential`).
  * **Lógica de Agregação:** O modelo usará o quantificador `Exists` (∃) para perguntar: "Existe evidência no tabuleiro para a aplicação de um Naked Single?". As pontuações desses especialistas são então passadas para uma camada final que decide qual heurística é a mais provável de ser útil.

### 8.1. Engenharia de Features e Geração de Dados

Primeiro, definimos as funções para criar nossos tensores de features e gerar o dataset. O dataset será rotulado com a heurística mais simples que se aplica a um determinado quebra-cabeça.

In [9]:
# Funções auxiliares para detecção de heurísticas no dataset
def generate_candidates(board):
    """Gera uma matriz de candidatos para cada célula do tabuleiro."""
    candidates = np.zeros((4, 4, 4), dtype=int) # Formato: (linha, coluna, número-1)
    for i in range(4):
        for j in range(4):
            if board[i, j] == 0:
                for num in range(1, 5):
                    if is_valid_placement(board, i, j, num):
                        candidates[i, j, num-1] = 1
    return candidates

def has_hidden_single(candidates):
    """Verifica se existe um 'Hidden Single' no tabuleiro."""
    for num in range(4):
        # Verifica linhas
        for i in range(4):
            if np.sum(candidates[i, :, num]) == 1:
                pos_j = np.argmax(candidates[i, :, num])
                if np.sum(candidates[i, pos_j, :]) > 1: return True
        # Verifica colunas
        for j in range(4):
            if np.sum(candidates[:, j, num]) == 1:
                pos_i = np.argmax(candidates[:, j, num])
                if np.sum(candidates[pos_i, j, :]) > 1: return True
        # Verifica blocos
        for bi in range(2):
            for bj in range(2):
                block = candidates[bi*2:bi*2+2, bj*2:bj*2+2, num]
                if np.sum(block) == 1:
                    r, c = np.unravel_index(np.argmax(block), block.shape)
                    abs_r, abs_c = bi*2+r, bj*2+c
                    if np.sum(candidates[abs_r, abs_c, :]) > 1: return True
    return False

def has_locked_candidate(candidates):
    """Verifica se existe um 'Locked Candidate' no tabuleiro."""
    for num in range(4):
        # Intersecção Bloco-Linha
        for i in range(4):
            cols = [j for j in range(4) if candidates[i,j,num] == 1]
            if len(cols) >= 2 and all(j//2 == cols[0]//2 for j in cols):
                bi, bj = i//2, cols[0]//2
                other_rows_in_block = [k for k in range(bi*2, bi*2+2) if k != i]
                if np.any(candidates[other_rows_in_block, :, num]): return True
        # Intersecção Bloco-Coluna
        for j in range(4):
            rows = [i for i in range(4) if candidates[i,j,num] == 1]
            if len(rows) >= 2 and all(i//2 == rows[0]//2 for i in rows):
                bi, bj = rows[0]//2, j//2
                other_cols_in_block = [k for k in range(bj*2, bj*2+2) if k != j]
                if np.any(candidates[:, other_cols_in_block, num]): return True
    return False

def create_feature_tensor(board):
    """Cria um tensor 3-canais para representar o estado do tabuleiro."""
    # Canal 0: Valores atuais (normalizados de 1-4 para 0-1)
    values = torch.from_numpy(board.astype(np.float32)).clone()
    values[values > 0] = (values[values > 0] - 1) / 3.0

    candidates = generate_candidates(board)
    # Canal 1: Contagem de candidatos por célula (normalizada)
    cand_count = torch.from_numpy(candidates.sum(axis=2)).float() / 4.0
    # Canal 2: Conflitos (simplificado, apenas indica se a célula está preenchida)
    conflicts = torch.from_numpy((board > 0).astype(np.float32))

    return torch.stack([values, cand_count, conflicts])

def generate_heuristic_dataset(num_samples):
    """Gera um dataset de (feature_tensor, label_da_heuristica)."""
    dataset = []
    heuristic_map = {"Naked": 0, "Hidden": 1, "Locked": 2, "Backtrack": 3}

    for _ in range(num_samples):
        board = generate_valid_board()
        puzzle = board.copy()
        # Remove entre 4 e 12 células para criar um quebra-cabeça
        empty_cells = random.randint(4, 12)
        positions = [(i,j) for i in range(4) for j in range(4)]
        random.shuffle(positions)
        for i, j in positions[:empty_cells]:
            puzzle[i,j] = 0

        features = create_feature_tensor(puzzle)
        candidates = generate_candidates(puzzle)
        cand_counts = candidates.sum(axis=2)

        # Atribui o rótulo com base na heurística mais simples aplicável
        if np.any(cand_counts == 1):
            label = heuristic_map["Naked"]
        elif has_hidden_single(candidates):
            label = heuristic_map["Hidden"]
        elif has_locked_candidate(candidates):
            label = heuristic_map["Locked"]
        else:
            label = heuristic_map["Backtrack"]

        dataset.append((features, torch.tensor(label, dtype=torch.long)))

    features_list, labels_list = zip(*dataset)
    return TensorDataset(torch.stack(features_list), torch.stack(labels_list))

### 8.2. Definição do Modelo Recomendador LTN

Aqui definimos a arquitetura do `LTNHeuristicRecommender4x4`.

In [10]:
class LTNHeuristicRecommender4x4(nn.Module):
    def __init__(self, feature_channels=3, num_heuristics=4):
        """
        Recomendador de heurísticas para Sudoku 4x4 usando LTN.

        Args:
            feature_channels: Número de canais de features (3 no nosso caso).
            num_heuristics: Número de heurísticas a classificar (4 no nosso caso).
        """
        super().__init__()

        # Predicados Especialistas: Cada um é uma pequena rede neural que aprende
        # a detectar o potencial para uma heurística específica.
        self.NakedSinglePotential = ltn.Predicate(nn.Sequential(
            nn.Linear(feature_channels, 8), nn.ReLU(), nn.Linear(8, 1), nn.Sigmoid()
        ))
        self.HiddenSinglePotential = ltn.Predicate(nn.Sequential(
            nn.Linear(feature_channels * 4, 16), nn.ReLU(), nn.Linear(16, 1), nn.Sigmoid()
        ))
        self.LockedCandidatePotential = ltn.Predicate(nn.Sequential(
            nn.Linear(feature_channels * 4, 16), nn.ReLU(), nn.Linear(16, 1), nn.Sigmoid()
        ))

        # Operadores Lógicos: Usamos 'Exists' para verificar se há evidência de uma heurística.
        self.Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")

        # Camada de Saída: Combina as pontuações dos especialistas para a classificação final.
        self.output_layer = nn.Linear(3, num_heuristics) # 3 scores de entrada, 4 saídas (logits)

    def forward(self, feature_tensor):
        """O forward pass que executa a lógica de recomendação."""
        # Prepara features para células, linhas, colunas e blocos
        cell_features = feature_tensor.view(feature_tensor.shape[0], -1).transpose(0, 1)
        row_features = feature_tensor.view(4, 3, 4).permute(0, 2, 1).reshape(4, -1)
        col_features = feature_tensor.view(4, 3, 4).permute(2, 0, 1).reshape(4, -1)
        block_features = []
        for i in range(2):
            for j in range(2):
                block = feature_tensor[:, i*2:(i+1)*2, j*2:(j+1)*2]
                block_features.append(block.reshape(-1))
        block_features = torch.stack(block_features)

        # Define as Variáveis LTN
        cells = ltn.Variable("cells", cell_features)
        rows = ltn.Variable("rows", row_features)
        cols = ltn.Variable("cols", col_features)
        blocks = ltn.Variable("blocks", block_features)

        # Avalia o potencial de cada heurística usando os predicados e o quantificador Exists
        naked_single_score = self.Exists(cells, self.NakedSinglePotential(cells))
        hidden_single_row = self.Exists(rows, self.HiddenSinglePotential(rows))
        hidden_single_col = self.Exists(cols, self.HiddenSinglePotential(cols))
        hidden_single_block = self.Exists(blocks, self.HiddenSinglePotential(blocks))
        # A pontuação geral para HiddenSingle é o máximo encontrado em qualquer linha, coluna ou bloco
        hidden_single_score = torch.max(torch.stack([
            hidden_single_row.value, hidden_single_col.value, hidden_single_block.value
        ]))
        locked_candidate_score = self.Exists(blocks, self.LockedCandidatePotential(blocks))

        # Combina as pontuações em um único tensor de features para a camada de saída
        heuristic_features = torch.stack([
            naked_single_score.value, hidden_single_score, locked_candidate_score.value
        ])

        # Gera os logits finais para a classificação
        logits = self.output_layer(heuristic_features)
        return logits

### 8.3. Treinamento do Recomendador

Agora, treinamos o modelo recomendador usando o dataset que geramos.

In [11]:
def train_recommender_model():
    """Função para treinar e retornar o modelo recomendador."""
    dataset = generate_heuristic_dataset(2000) # Gerar um dataset maior
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = LTNHeuristicRecommender4x4().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    print("Iniciando o treinamento do Recomendador de Heurísticas...")
    for epoch in range(30): # Número de épocas de treinamento
        total_loss = 0
        correct = 0
        total = 0

        for features_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            outputs_list = []
            # O modelo atual processa uma amostra por vez.
            # Este loop itera sobre o batch para processar cada amostra individualmente.
            for i in range(features_batch.size(0)):
                features_single = features_batch[i].to(device)
                output_single = model(features_single)
                outputs_list.append(output_single)
            outputs = torch.stack(outputs_list)
            labels_batch = labels_batch.to(device)

            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels_batch.size(0)
            correct += (predicted == labels_batch).sum().item()

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 5 == 0:
            print(f"Época {epoch+1:02d}, Perda: {avg_loss:.4f}, Acurácia: {accuracy:.2f}%")

    print("Treinamento concluído!")
    return model

# Executa o treinamento
recommender_model = train_recommender_model()

Iniciando o treinamento do Recomendador de Heurísticas...
Época 05, Perda: 1.3355, Acurácia: 0.00%
Época 10, Perda: 0.9055, Acurácia: 80.65%
Época 15, Perda: 0.7315, Acurácia: 80.65%
Época 20, Perda: 0.6839, Acurácia: 80.65%
Época 25, Perda: 0.6645, Acurácia: 80.65%
Época 30, Perda: 0.6446, Acurácia: 80.65%
Treinamento concluído!


### 8.4. Execução: Usando o Recomendador

Finalmente, vamos testar o recomendador treinado em um novo sudoku para ver qual estratégia ele sugere.

In [12]:
# Exemplo de uso
if __name__ == "__main__":
    # 2. Ler o tabuleiro de um arquivo CSV
    try:
        # Especifique o caminho para o seu arquivo CSV
        csv_file_path = "sudoku_4x4_board.csv"
        puzzle_df = pd.read_csv(csv_file_path, header=None)

        # Converter o DataFrame do pandas para um array NumPy, que é o formato esperado
        puzzle = puzzle_df.values.astype(int)

    except FileNotFoundError:
        print(f"Erro: Arquivo '{csv_file_path}' não encontrado.")
        print("Por favor, crie o arquivo CSV com um tabuleiro 4x4 ou corrija o caminho.")
        exit() # Encerra o script se o arquivo não for encontrado

    # 3. Criar o tensor de features a partir do tabuleiro lido
    features = create_feature_tensor(puzzle)

    # 4. Fazer a recomendação com o modelo treinado
    recommender_model.eval() # Coloca o modelo em modo de avaliação
    with torch.no_grad(): # Desativa o cálculo de gradientes para inferência
        heuristic_logits = recommender_model(features)

    # 5. Processar e exibir os resultados
    heuristic_scores = torch.softmax(heuristic_logits, dim=0)
    heuristic_names = ["Naked Single", "Hidden Single", "Locked Candidate", "Backtracking"]

    print("Tabuleiro lido do arquivo:")
    print(puzzle)

    print("\nRecomendação de heurísticas:")
    for name, score in zip(heuristic_names, heuristic_scores):
        print(f"{name}: {score.item():.4f}")

    recommended_idx = torch.argmax(heuristic_scores).item()
    recommended = heuristic_names[recommended_idx]
    print(f"\nHeurística mais recomendada: {recommended}")

Tabuleiro lido do arquivo:
[[1 0 0 4]
 [0 2 3 0]
 [0 4 1 0]
 [2 0 0 3]]

Recomendação de heurísticas:
Naked Single: 0.0338
Hidden Single: 0.8032
Locked Candidate: 0.0729
Backtracking: 0.0901

Heurística mais recomendada: Hidden Single
